# Basic Trend following algorithms

## **Research Paper: Trend Following Algorithms for Technical Trading in Stock Market by Fong, Tai, Si**

http://www.jetwi.us/index.php?m=content&c=index&a=show&catid=158&id=890

Two rules of their model: P and Q

- P: The amount of trend in an upward direction for the trade to be opened (when to enter the market)
- Q: The amount of opposite trend after a pivotal turn that will cause an opened position to close. (When to exit the trade)

Open when t > P, close when t < Q. **Price(t)**


$$ EMA_t = (price_t - EMA_{t-1} \times \frac{2}{n+1}) + EMA_{t-1}  $$
- price(t) is the current price at time t
- n is the number of periods (by aggregation type) for intra-day tradings
- t is any given market time



Need to derive P and Q empirically. Use cross validation and allow them to dynamically change.

How do you find the buying and selling positions in real-time if P and Q change? Use the Relative Strength Index (RSI)

$$ RSI_t = 100 - \frac{100}{1+RS_{t}} $$

$$ RS_t = \frac{AU_t}{AD_t} $$

$$ AU_t = \frac{Up_t + Up_{(t-1)} + ... + Up_{(t - n + 1)}}{n} $$

$$ AD_t = \frac{Down_t + Down_{(t-1)} + ... + Down_{(t - n + 1)}}{n} $$

AD is the average price upwards, AD is the average price downwards and n is the number of RSI periods. Typically 14 but used cross-validation

**RSI**
- Scale of 0 to 100. If it approaches 70, the asset may be overvalued and is a good time to pullback
- If it approaches 30, likely to be oversold and may be undervalued
- Above 50 = bullish, below 50 = bearish. Below 30 is a bearish danger zone

**How does a position open under this algorithm?**
- For a long position:
    - Price is advancing
    - RSI(t) is greater than EMA(RSI(t))
    - EMA(RSI(t)) is less than 40 or greater than 60
- For a short position:
    - Price is declining
    - RSI(t) is less than EMA(RSI(t))
    - EMA(RSI(t)) is less than 40 or greater than 60
    
**This model could be bad in assets with high volatility. When fluctuation goes about 45%, pull out**

$$ Fluctuate_t = (COS_e \times C \times R_t) + B $$

In [552]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath('../'))
from helper_funcs import get_data, convert_epoch_to_datetime

In [553]:
DB_PATH = os.environ.get('DB_PATH')

btc_df = get_data('BTC', 'USDT', DB_PATH)
btc_df = convert_epoch_to_datetime(btc_df, 'close_time')
btc_df['close'] = btc_df['close'].astype(float)

In [554]:
last_btc_halving = '2021-05-11'
btc_df[btc_df['date'] >= last_btc_halving]
btc_df.set_index('date', inplace=True)
one_day_df = btc_df[-289:]

In [555]:
btc_df['EMA'] = (btc_df['close'].diff(-1) * -1) * (2 / 289) + btc_df['close']

In [556]:
first_value_of_each_day = btc_df.groupby(btc_df.index.date).apply(lambda x: x.iloc[0])

In [557]:
def rsi(df, periods = 14, ema = True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
	    # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [558]:
btc_df.index = btc_df.index.floor('min')
first_value_of_each_day.index = pd.to_datetime(first_value_of_each_day.index)
first_value_of_each_day['RSI'] = rsi(first_value_of_each_day)

In [559]:
btc_df.index = btc_df.index.map(lambda x: x.date())
first_value_of_each_day.index = first_value_of_each_day.index.map(lambda x: x.date())

In [560]:
btc_df['RSI'] = btc_df.join(first_value_of_each_day['RSI'])['RSI'].values

In [561]:
btc_df

,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,ignore,EMA,RSI
date,,,,,,,,,,,,
2017-08-17,4261.48,4280.56,4261.48,4261.48,2.189061,9333.6209622,9,0.489061,2089.1049622,7960.37529526,4261.480000,NaN
2017-08-17,4261.48,4261.48,4261.48,4261.48,0.0,0.0,0,0.0,0.0,7959.62629202,4261.480000,NaN
2017-08-17,4261.48,4261.48,4261.48,4261.48,0.0,0.0,0,0.0,0.0,7958.41741512,4261.480000,NaN
2017-08-17,4261.48,4264.88,4261.48,4261.48,0.484666,2065.44782868,9,0.085455,364.2181364,7958.34896534,4261.513287,NaN
2017-08-17,4264.88,4266.29,4264.88,4266.29,2.32857,9931.16112417,11,1.546491,6595.68803865,7953.96934947,4266.256505,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-10,31820.70000000,31924.49000000,31771.84000000,31831.24,592.32911000,18854629.20816770,7162,217.81762000,6933562.39801990,0,31831.042284,24.628943
2022-05-10,31831.24000000,31846.99000000,31762.78000000,31802.67,481.34955000,15310650.36983240,7078,196.80650000,6259062.20593220,0,31802.333945,24.628943
2022-05-10,31802.66000000,31850.00000000,31716.42000000,31754.11,460.41145000,14629993.63707830,7394,200.96762000,6385450.93289980,0,31753.813599,24.628943
